### Large Data Files Processing with both DASK and Pandas

- Files are imported using DASK, a library which is intentended to be a clone of pandas that supports multiprocessing. The library is incomplete though and as the task counts of the data set builds up, running operations becomes increasingly slower and more stressful on the kernel so the minimal time is spent using it.
- The DASK framework enabled us to drop the memory size of the dataset so that it could run in ran. This was achieved through the dropping of unuseful columns, factorizinig of column values and reducing the memory sizes of columns to their loweest necessary data size. (eg int64 columns who only needed 32bit to store the largest and smallest values were converted to int32)
- Many hanges to the large data set in this notebook were done in tandem with insights taken from smaller notebooks which worked on segmented datasets of particular bus routes.
- This notebook's structure and cell numbering does not have a perfect linear structure due to RAM preserving adjustments which needed to be made in order to deal with the issues that arose with the UCD server's performance in the altter half of the semester.
- Although it would have been possible should the UCD server's performance have stayed strong, we chose to apply the more computationally taxing cleaning techniques to the files after they had already been broken up.


In [1]:
import pandas as pd
import dask.dataframe as dd 
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
import gc
import psutil
import multiprocessing
import time
import os
import signal
from os import getpid
from sys import argv, exit

In [2]:
psutil.virtual_memory()

svmem(total=67481169920, available=26697043968, percent=60.4, used=32030101504, free=17713520640, active=29675589632, inactive=18249072640, buffers=1880395776, cached=15857152000, shared=8072052736, slab=1142431744)

In [3]:
psutil.users()

[suser(name='team9', terminal='pts/19', host='95.44.114.87', started=1660195456.0, pid=29053),
 suser(name='team9', terminal='pts/22', host='95.44.114.87', started=1660195584.0, pid=29453),
 suser(name='team10', terminal='pts/31', host='tmux(22475).%0', started=1657274368.0, pid=22475)]

In [4]:
cd tmp

/home/team9/tmp


In [5]:
ls

data/  line_46A_2-Copy1.csv


In [6]:
cd data

/home/team9/tmp/data


In [10]:
weather = pd.read_csv('Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv')

In [11]:
weather.shape

(27186, 28)

In [13]:
weather

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,1483228800,2017-01-01 00:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,1.78,...,NaN,2.30,NaN,NaN,NaN,75,501,Rain,moderate rain,10n
1,1483232400,2017-01-01 01:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,2.28,...,NaN,1.51,NaN,NaN,NaN,75,501,Rain,moderate rain,10n
2,1483236000,2017-01-01 02:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,5.39,9999.0,4.35,2.28,...,NaN,0.64,NaN,NaN,NaN,75,500,Rain,light rain,10n
3,1483239600,2017-01-01 03:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,4.39,9999.0,3.36,0.04,...,NaN,0.17,NaN,NaN,NaN,75,500,Rain,light rain,10n
4,1483243200,2017-01-01 04:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,4.39,9999.0,2.42,0.04,...,NaN,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27181,1577818800,2019-12-31 19:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,9.33,9999.0,6.42,9.33,...,2.68,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n
27182,1577822400,2019-12-31 20:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,8.81,9999.0,6.09,7.02,...,NaN,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
27183,1577826000,2019-12-31 21:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,7.86,9999.0,5.33,7.86,...,1.79,NaN,NaN,NaN,NaN,90,804,Clouds,overcast clouds,04n
27184,1577829600,2019-12-31 22:00:00 +0000 UTC,0,Custom location,53.345035,-6.267261,7.49,9999.0,4.09,7.49,...,1.79,NaN,NaN,NaN,NaN,75,803,Clouds,broken clouds,04n


In [14]:
weather.nunique()

dt                     26280
dt_iso                 26280
timezone                   2
city_name                  1
lat                        1
lon                        1
temp                    2009
visibility                45
dew_point               1929
feels_like              2550
temp_min                 672
temp_max                 919
pressure                  74
sea_level                  0
grnd_level                 0
humidity                  73
wind_speed               101
wind_deg                 358
wind_gust                 85
rain_1h                  281
rain_3h                    0
snow_1h                   49
snow_3h                    0
clouds_all                17
weather_id                27
weather_main              10
weather_description       29
weather_icon              17
dtype: int64

- most look unhelpful
- keeping to categorical data
- weather description, weather icon, temperature, visibility, wind speed, temperature
- will condense the categories down for better predictions
- i think i can ignore the timezone issue because the weather icon says whether it is day or night and it is only 1 hour difference and there is a timezone element in the dt iso column
- there are not an equal amount of datetimes as total columns so must remove duplicates
- weather icon dropoped after model testing

In [15]:
weather = weather.drop(["dt_iso", "temp_min", "weather_id", "clouds_all", "snow_3h", "snow_1h", "rain_3h", "rain_1h", "wind_gust", "wind_deg", "humidity", "grnd_level", "sea_level", "pressure", "temp_max", "feels_like", "dew_point", "lon", "lat", "city_name", "timezone"], axis=1)

In [16]:
weather

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5.39,9999.0,5.10,Rain,moderate rain,10n
1,1483232400,5.39,9999.0,4.10,Rain,moderate rain,10n
2,1483236000,5.39,9999.0,4.10,Rain,light rain,10n
3,1483239600,4.39,9999.0,6.20,Rain,light rain,10n
4,1483243200,4.39,9999.0,6.20,Clouds,broken clouds,04n
...,...,...,...,...,...,...,...
27181,1577818800,9.33,9999.0,0.45,Clouds,broken clouds,04n
27182,1577822400,8.81,9999.0,3.10,Clouds,overcast clouds,04n
27183,1577826000,7.86,9999.0,0.45,Clouds,overcast clouds,04n
27184,1577829600,7.49,9999.0,0.45,Clouds,broken clouds,04n


In [17]:
print(weather["temp"].max())
print(weather["temp"].min())

26.92
-4.61


In [18]:
weather.dtypes

dt                       int64
temp                   float64
visibility             float64
wind_speed             float64
weather_main            object
weather_description     object
weather_icon            object
dtype: object

In [19]:
dupli_epoch = weather[weather.duplicated(['dt'])]


In [20]:
dupli_epoch

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
135,1483711200,11.39,9000.0,6.70,Rain,light rain,10d
137,1483714800,11.39,9999.0,6.20,Rain,light rain,10d
139,1483718400,11.92,9999.0,6.70,Rain,light rain,10d
199,1483930800,9.39,9999.0,9.80,Drizzle,light intensity drizzle,09n
655,1485568800,6.00,9999.0,2.60,Rain,light rain,10n
...,...,...,...,...,...,...,...
27002,1577188800,7.48,9000.0,0.89,Rain,light rain,10d
27062,1577401200,8.62,8000.0,3.10,Rain,light rain,10n
27064,1577404800,8.43,8000.0,2.10,Rain,light rain,10n
27066,1577408400,8.70,5000.0,3.10,Rain,light rain,10n


In [21]:
dupliexamine_epoch = weather.loc[weather['dt'] == 1483711200]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483718400]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483711200]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1483930800]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1577404800]
print(dupliexamine_epoch)
dupliexamine_epoch = weather.loc[weather['dt'] == 1577412000]
print(dupliexamine_epoch)

             dt   temp  visibility  wind_speed weather_main  \
134  1483711200  11.39      9000.0         6.7      Drizzle   
135  1483711200  11.39      9000.0         6.7         Rain   

              weather_description weather_icon  
134  light intensity drizzle rain          09d  
135                    light rain          10d  
             dt   temp  visibility  wind_speed weather_main  \
138  1483718400  11.92      9999.0         6.7      Drizzle   
139  1483718400  11.92      9999.0         6.7         Rain   

              weather_description weather_icon  
138  light intensity drizzle rain          09d  
139                    light rain          10d  
             dt   temp  visibility  wind_speed weather_main  \
134  1483711200  11.39      9000.0         6.7      Drizzle   
135  1483711200  11.39      9000.0         6.7         Rain   

              weather_description weather_icon  
134  light intensity drizzle rain          09d  
135                    light rain     

- duplicates are all basically the same with slight variations in the final 2 columns so are not worth keeping in

In [22]:
weather = weather.drop_duplicates(subset=['dt'])


In [23]:
weather.loc[130:140]
#check to see drop happnened
# 135 should be gone

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
130,1483696800,10.44,8000.0,6.0,Rain,light rain,10d
131,1483700400,11.39,9999.0,4.6,Rain,light rain,10d
132,1483704000,11.39,9999.0,6.2,Clouds,broken clouds,04d
133,1483707600,11.39,9999.0,7.2,Rain,light rain,10d
134,1483711200,11.39,9000.0,6.7,Drizzle,light intensity drizzle rain,09d
136,1483714800,11.39,9999.0,6.2,Drizzle,light intensity drizzle rain,09d
138,1483718400,11.92,9999.0,6.7,Drizzle,light intensity drizzle rain,09d
140,1483722000,11.92,9999.0,6.2,Drizzle,light intensity drizzle,09n


In [24]:
weather=weather.reset_index(drop=True)

In [25]:
weather.wind_speed.max()

22.1

In [26]:
weather.wind_speed.min()

0.0

- must be in miles per hour

In [27]:
#  unfactorised one meant for analysing the features
#weather.to_csv('./weather_trips.csv')

In [28]:
# function for rounding 
def round_int(x, base):
    return int(base * round(float(x)/base))

weather['temp'] = weather['temp'].apply(lambda x: round_int(x, base=1))


In [29]:
weather.head()

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5,9999.0,5.1,Rain,moderate rain,10n
1,1483232400,5,9999.0,4.1,Rain,moderate rain,10n
2,1483236000,5,9999.0,4.1,Rain,light rain,10n
3,1483239600,4,9999.0,6.2,Rain,light rain,10n
4,1483243200,4,9999.0,6.2,Clouds,broken clouds,04n


In [30]:
weather.temp.max()

27

In [31]:
weather.temp.min()

-5

In [32]:
weather.head(10)

,dt,temp,visibility,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5,9999.0,5.1,Rain,moderate rain,10n
1,1483232400,5,9999.0,4.1,Rain,moderate rain,10n
2,1483236000,5,9999.0,4.1,Rain,light rain,10n
3,1483239600,4,9999.0,6.2,Rain,light rain,10n
4,1483243200,4,9999.0,6.2,Clouds,broken clouds,04n
5,1483246800,3,9999.0,6.2,Clouds,scattered clouds,03n
6,1483250400,2,9999.0,5.7,Clouds,few clouds,02n
7,1483254000,2,9999.0,6.7,Clouds,few clouds,02n
8,1483257600,2,9999.0,6.2,Clouds,few clouds,02n
9,1483261200,3,9999.0,5.7,Clouds,few clouds,02d


In [33]:
weather.isnull().sum()

dt                      0
temp                    0
visibility             45
wind_speed              0
weather_main            0
weather_description     0
weather_icon            0
dtype: int64

- set as minus 1 so that rows wont need to be dropped as it may be an issue with joining if certain times are missing

In [34]:
weather['visibility'] = weather['visibility'].fillna(-1)
weather.isnull().sum()

dt                     0
temp                   0
visibility             0
wind_speed             0
weather_main           0
weather_description    0
weather_icon           0
dtype: int64

In [35]:
weather['visibility'].nunique()

46

- using visibility has potential problems as it's not clear yet whether the data we will use for real time will match the format. 
- after check with the results in the data analysis the feature was not sufficiently important to justify keeping.

In [36]:
weather = weather.drop(["visibility"], axis=1)

In [37]:
weather.nunique()

dt                     26280
temp                      33
wind_speed               101
weather_main              10
weather_description       29
weather_icon              17
dtype: int64

In [38]:
max_wind = weather["wind_speed"].max()
min_wind = weather["wind_speed"].min()

print(max_wind)
print(min_wind)

22.1
0.0


In [39]:
weather["wind_speed"] = weather["wind_speed"].apply(lambda x: round_int(x, base=2))

In [40]:
max_wind = weather["wind_speed"].max()
min_wind = weather["wind_speed"].min()

print(max_wind)
print(min_wind)

22
0


In [41]:
#weather["wind_speed"] = pd.factorize(weather["wind_speed"], sort=True)[0]

In [42]:
weather.head(10)

,dt,temp,wind_speed,weather_main,weather_description,weather_icon
0,1483228800,5,6,Rain,moderate rain,10n
1,1483232400,5,4,Rain,moderate rain,10n
2,1483236000,5,4,Rain,light rain,10n
3,1483239600,4,6,Rain,light rain,10n
4,1483243200,4,6,Clouds,broken clouds,04n
5,1483246800,3,6,Clouds,scattered clouds,03n
6,1483250400,2,6,Clouds,few clouds,02n
7,1483254000,2,6,Clouds,few clouds,02n
8,1483257600,2,6,Clouds,few clouds,02n
9,1483261200,3,6,Clouds,few clouds,02d


In [43]:
weather['weather_main'] = weather['weather_main'].apply(lambda x: 1 if (x == "Rain" or x == "Mist" or x == "Thunderstorm") else 0)

- dropping the weather icon and weather description
    - after model testing

In [44]:
weather = weather.drop(["weather_description", "weather_icon"], axis=1)

In [45]:
weather

,dt,temp,wind_speed,weather_main
0,1483228800,5,6,1
1,1483232400,5,4,1
2,1483236000,5,4,1
3,1483239600,4,6,1
4,1483243200,4,6,0
...,...,...,...,...
26275,1577818800,9,0,0
26276,1577822400,9,4,0
26277,1577826000,8,0,0
26278,1577829600,7,0,0


In [46]:
weather["weather_main"] = pd.factorize(weather["weather_main"], sort=True)[0]

In [47]:
weather.head(5)

,dt,temp,wind_speed,weather_main
0,1483228800,5,6,1
1,1483232400,5,4,1
2,1483236000,5,4,1
3,1483239600,4,6,1
4,1483243200,4,6,0


In [48]:
weather.dtypes

dt              int64
temp            int64
wind_speed      int64
weather_main    int64
dtype: object

In [49]:
# makes date column for join
weather['date'] = pd.to_datetime(weather['dt'], unit='s')
weather

,dt,temp,wind_speed,weather_main,date
0,1483228800,5,6,1,2017-01-01 00:00:00
1,1483232400,5,4,1,2017-01-01 01:00:00
2,1483236000,5,4,1,2017-01-01 02:00:00
3,1483239600,4,6,1,2017-01-01 03:00:00
4,1483243200,4,6,0,2017-01-01 04:00:00
...,...,...,...,...,...
26275,1577818800,9,0,0,2019-12-31 19:00:00
26276,1577822400,9,4,0,2019-12-31 20:00:00
26277,1577826000,8,0,0,2019-12-31 21:00:00
26278,1577829600,7,0,0,2019-12-31 22:00:00


In [50]:
# only interested in weather from 2018
weather = weather.loc[weather['date']<'2019']

In [51]:
weather = weather.loc[weather['date']>'2018']

In [52]:
weather=weather.reset_index(drop=True)
weather

,dt,temp,wind_speed,weather_main,date
0,1514768400,4,12,1,2018-01-01 01:00:00
1,1514772000,5,12,0,2018-01-01 02:00:00
2,1514775600,5,12,0,2018-01-01 03:00:00
3,1514779200,5,12,0,2018-01-01 04:00:00
4,1514782800,5,10,0,2018-01-01 05:00:00
...,...,...,...,...,...
8754,1546282800,10,2,0,2018-12-31 19:00:00
8755,1546286400,10,0,0,2018-12-31 20:00:00
8756,1546290000,10,0,0,2018-12-31 21:00:00
8757,1546293600,10,2,0,2018-12-31 22:00:00


In [53]:
weather["date"] = weather["date"].astype("datetime64")
weather['DAYOFWEEK'] = weather['date'].dt.weekday

In [54]:
weather

,dt,temp,wind_speed,weather_main,date,DAYOFWEEK
0,1514768400,4,12,1,2018-01-01 01:00:00,0
1,1514772000,5,12,0,2018-01-01 02:00:00,0
2,1514775600,5,12,0,2018-01-01 03:00:00,0
3,1514779200,5,12,0,2018-01-01 04:00:00,0
4,1514782800,5,10,0,2018-01-01 05:00:00,0
...,...,...,...,...,...,...
8754,1546282800,10,2,0,2018-12-31 19:00:00,0
8755,1546286400,10,0,0,2018-12-31 20:00:00,0
8756,1546290000,10,0,0,2018-12-31 21:00:00,0
8757,1546293600,10,2,0,2018-12-31 22:00:00,0


In [57]:
weather.nunique()

dt              8759
temp              33
wind_speed        10
weather_main       2
date            8759
DAYOFWEEK          7
dtype: int64

In [58]:
#weather.to_csv('./weather_factorised.csv')

# Bus Data

In [9]:
psutil.virtual_memory()

svmem(total=67481169920, available=19750899712, percent=70.7, used=38974631936, free=11555160064, active=34063101952, inactive=20157714432, buffers=214896640, cached=16736481280, shared=8072019968, slab=947777536)

In [7]:
%%time
trips = dd.read_csv('rt_trips_DB_2018.txt', sep=";", dtype={'DATASOURCE': 'category', 'DAYOFSERVICE': 'object', 'TRIPID': 'int32', 'LINEID': 'category', 'ROUTEID': 'category', 'DIRECTION': 'category', 'PLANNEDTIME_ARR': 'float64', 'PLANNEDTIME_DEP': 'float64', 'ACTUALTIME_ARR': 'float64', 'ACTUALTIME_DEP': 'float64', 'BASIN': 'category', 'TENDERLOT': 'category', 'SUPPRESSED': 'category', 'JUSTIFICATIONID': 'string', 'LASTUPDATE': 'string', 'NOTE': 'string'})
#trips
# ACTUALTIME_DEP had to be float64 because of issue with the data to do with null values

CPU times: user 33.6 ms, sys: 2.57 ms, total: 36.2 ms
Wall time: 213 ms


In [8]:
psutil.virtual_memory()

svmem(total=67481169920, available=26704748544, percent=60.4, used=32022335488, free=18019004416, active=29652357120, inactive=17970089984, buffers=1880383488, cached=15559446528, shared=8072052736, slab=1142063104)

In [9]:
print(type(trips))

<class 'dask.dataframe.core.DataFrame'>


In [10]:
trips

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
npartitions=3,,,,,,,,,,,,,,,,
,category[unknown],object,int32,category[unknown],category[unknown],category[unknown],float64,float64,float64,float64,category[unknown],category[unknown],category[unknown],string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
trips.map_partitions(type).compute()

0    <class 'pandas.core.frame.DataFrame'>
1    <class 'pandas.core.frame.DataFrame'>
2    <class 'pandas.core.frame.DataFrame'>
dtype: object

In [15]:
psutil.virtual_memory()

svmem(total=67481169920, available=19540074496, percent=71.0, used=39186141184, free=11125739520, active=34275926016, inactive=20377067520, buffers=214876160, cached=16954413056, shared=8072019968, slab=946151424)

In [11]:
# converts to pandas
trips = trips.compute()

In [12]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182637 entries, 0 to 727498
Data columns (total 16 columns):
 #   Column           Dtype   
---  ------           -----   
 0   DATASOURCE       category
 1   DAYOFSERVICE     object  
 2   TRIPID           int32   
 3   LINEID           category
 4   ROUTEID          category
 5   DIRECTION        category
 6   PLANNEDTIME_ARR  float64 
 7   PLANNEDTIME_DEP  float64 
 8   ACTUALTIME_ARR   float64 
 9   ACTUALTIME_DEP   float64 
 10  BASIN            category
 11  TENDERLOT        category
 12  SUPPRESSED       category
 13  JUSTIFICATIONID  string  
 14  LASTUPDATE       string  
 15  NOTE             string  
dtypes: category(7), float64(4), int32(1), object(1), string(3)
memory usage: 177.0+ MB


In [13]:
psutil.virtual_memory()

svmem(total=67481169920, available=26234757120, percent=61.1, used=32492371968, free=17316241408, active=30121365504, inactive=18201718784, buffers=1880387584, cached=15792168960, shared=8072052736, slab=1142292480)

In [16]:
trips.head(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0,BasDef,NaN,NaN,<NA>,28-FEB-18 12:05:11,",2448968,"


In [19]:
trips.DIRECTION.value_counts()

2     1100273
1     1082364
Name: DIRECTION, dtype: int64

In [20]:
trips.duplicated().sum()

0

In [23]:
trips.tail(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
727496,DB,14-MAY-18 00:00:00,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",3077688,"
727497,DB,14-MAY-18 00:00:00,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",3208841,"
727498,DB,14-MAY-18 00:00:00,6765012,27,27_19,1,47722.0,41700.0,47508.0,41642.0,BasDef,NaN,NaN,<NA>,26-JUN-18 09:13:13,",2960092,"


- multiple empty columns
- multiple probable useless ones
    - "note", "last update", "basin", "LASTUPDATE"

In [14]:
trips = trips.drop({'TENDERLOT', "SUPPRESSED", "JUSTIFICATIONID", "DATASOURCE", "LASTUPDATE", "NOTE","BASIN"}, axis=1)


In [35]:
trips

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261.0,54420.0,58463.0,54443.0
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383.0,81600.0,84682.0,81608.0
...,...,...,...,...,...,...,...,...,...
727494,14-MAY-18 00:00:00,6765849,123,123_36,2,61560.0,57840.0,61365.0,57859.0
727495,14-MAY-18 00:00:00,6765469,75,75_17,1,53416.0,48600.0,NaN,48823.0
727496,14-MAY-18 00:00:00,6765486,33D,33D_62,2,29460.0,26400.0,29904.0,NaN
727497,14-MAY-18 00:00:00,6764987,70,70_60,1,65277.0,60600.0,66341.0,NaN


In [36]:
trips.head(3)

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245.0,84600.0,87524.0,84600.0
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517.0,26460.0,32752.0,NaN
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512.0,32100.0,36329.0,32082.0


In [15]:
#changes the formats and created column to merge with that used less memory
trips["DAYOFSERVICE"] = trips["DAYOFSERVICE"].astype("datetime64")
trips["DOY"] = trips["DAYOFSERVICE"].dt.dayofyear

trips["DOY"] = trips["DOY"].astype("int16")

trips = trips.drop("DAYOFSERVICE", axis=1)

In [16]:
trips.DOY.nunique()

360

- 5 days missing

In [7]:
psutil.virtual_memory()

svmem(total=67481169920, available=26694668288, percent=60.4, used=32032460800, free=17711017984, active=29680484352, inactive=18249068544, buffers=1880395776, cached=15857295360, shared=8072052736, slab=1142521856)

In [8]:
%%time
leaveTimes = dd.read_csv('rt_leavetimes_DB_2018.txt', sep=";", dtype={'DATASOURCE': 'category', 'DAYOFSERVICE': 'object', 'PROGRNUMBER': 'int8', 'STOPPOINTID': 'int16','TRIPID': 'int32', 'LINEID': 'category', 'ROUTEID': 'category', 'VEHICLEID': 'category', 'PLANNEDTIME_ARR': 'int32', 'PLANNEDTIME_DEP': 'int32', 'ACTUALTIME_ARR': 'int32', 'ACTUALTIME_DEP': 'int32','PASSENGERSIN': 'category', 'PASSENGERSOUT': 'category', 'DISTANCE': 'category', 'SUPPRESSED': 'category', 'JUSTIFICATIONID': 'string', 'LASTUPDATE': 'string', 'NOTE': 'string'})

CPU times: user 51.4 ms, sys: 3.74 ms, total: 55.2 ms
Wall time: 103 ms


In [9]:
psutil.virtual_memory()

svmem(total=67481169920, available=26691756032, percent=60.4, used=32035364864, free=17707462656, active=29683171328, inactive=18249711616, buffers=1880399872, cached=15857942528, shared=8072052736, slab=1142521856)

In [10]:
leaveTimes

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
npartitions=176,,,,,,,,,,,,,,,,,,
,category[unknown],object,int32,int8,int16,int32,int32,int32,int32,category[unknown],float64,category[unknown],category[unknown],category[unknown],category[unknown],string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
leaveTimes = leaveTimes.drop({"PLANNEDTIME_DEP", "ACTUALTIME_DEP", "DISTANCE", "VEHICLEID", "SUPPRESSED", "JUSTIFICATIONID", "DATASOURCE", "NOTE", "PASSENGERSOUT", "PASSENGERSIN", "PASSENGERS", "LASTUPDATE" }, axis=1)

In [12]:
leaveTimes.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR
0,01-JAN-18 00:00:00,5972116,12,119,48030,48012
1,01-JAN-18 00:00:00,5966674,12,119,54001,54023
2,01-JAN-18 00:00:00,5959105,12,119,60001,59955


In [13]:
leaveTimes

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR
npartitions=176,,,,,,
,object,int32,int8,int16,int32,int32
,...,...,...,...,...,...
...,...,...,...,...,...,...
,...,...,...,...,...,...
,...,...,...,...,...,...


In [14]:
leaveTimes["DAYOFSERVICE"] = leaveTimes["DAYOFSERVICE"].astype("datetime64")


In [15]:
leaveTimes["DOY"] = leaveTimes["DAYOFSERVICE"].dt.dayofyear


In [16]:
leaveTimes.head(3)

,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY
0,2018-01-01,5972116,12,119,48030,48012,1
1,2018-01-01,5966674,12,119,54001,54023,1
2,2018-01-01,5959105,12,119,60001,59955,1


In [17]:
# puts back into smaller memory size
leaveTimes["DAYOFSERVICE"] = leaveTimes["DAYOFSERVICE"].astype("category")
leaveTimes["DOY"] = leaveTimes["DOY"].astype("int16")

In [18]:
leaveTimes.dtypes

DAYOFSERVICE       category
TRIPID                int32
PROGRNUMBER            int8
STOPPOINTID           int16
PLANNEDTIME_ARR       int32
ACTUALTIME_ARR        int32
DOY                   int16
dtype: object

In [19]:
psutil.virtual_memory()

svmem(total=67481169920, available=26559164416, percent=60.6, used=32167976960, free=17510879232, active=29879955456, inactive=18249281536, buffers=1880399872, cached=15921913856, shared=8072052736, slab=1142366208)

In [20]:
%%time
# drops DAYOFSERVICE and switches to pandas
# better to do early before tasks count gets too big it can make the computation take longer
leaveTimes = leaveTimes.drop("DAYOFSERVICE", axis=1).compute()

CPU times: user 6min 35s, sys: 1min 8s, total: 7min 44s
Wall time: 3min 45s


In [21]:
# clears bad cache from ram
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (10 entries)


In [22]:
psutil.virtual_memory()

svmem(total=67481169920, available=19450544128, percent=71.2, used=39272349696, free=7935504384, active=35673001984, inactive=22296117248, buffers=1215574016, cached=19057741824, shared=8072048640, slab=866537472)

In [23]:
leaveTimes

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY
0,5972116,12,119,48030,48012,1
1,5966674,12,119,54001,54023,1
2,5959105,12,119,60001,59955,1
3,5966888,12,119,58801,58771,1
4,5965960,12,119,56401,56309,1
...,...,...,...,...,...,...
664446,8588153,78,4383,28605,28998,365
664447,8587459,78,4383,22695,23247,365
664448,8586183,78,4383,51481,52237,365
664449,8589374,23,7053,53659,53525,365


In [62]:
len(leaveTimes)

116949113

In [63]:
leaveTimes.tail()

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY
664446,8588153,78,4383,28605,28998,365
664447,8587459,78,4383,22695,23247,365
664448,8586183,78,4383,51481,52237,365
664449,8589374,23,7053,53659,53525,365
664450,8589372,24,2088,46383,46315,365


In [24]:
%%time
print(leaveTimes.reset_index(drop=True).tail())

            TRIPID  PROGRNUMBER  STOPPOINTID  PLANNEDTIME_ARR  ACTUALTIME_ARR  \
116949108  8588153           78         4383            28605           28998   
116949109  8587459           78         4383            22695           23247   
116949110  8586183           78         4383            51481           52237   
116949111  8589374           23         7053            53659           53525   
116949112  8589372           24         2088            46383           46315   

           DOY  
116949108  365  
116949109  365  
116949110  365  
116949111  365  
116949112  365  
CPU times: user 2.13 s, sys: 33.5 s, total: 35.6 s
Wall time: 35.8 s


In [25]:
%%time
leaveTimes = leaveTimes.reset_index(drop=True)

CPU times: user 2.22 s, sys: 14.5 s, total: 16.7 s
Wall time: 16.8 s


In [65]:
leaveTimes

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY
0,5972116,12,119,48030,48012,1
1,5966674,12,119,54001,54023,1
2,5959105,12,119,60001,59955,1
3,5966888,12,119,58801,58771,1
4,5965960,12,119,56401,56309,1
...,...,...,...,...,...,...
116949108,8588153,78,4383,28605,28998,365
116949109,8587459,78,4383,22695,23247,365
116949110,8586183,78,4383,51481,52237,365
116949111,8589374,23,7053,53659,53525,365


- data is intact

In [26]:
leaveTimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116949113 entries, 0 to 116949112
Data columns (total 6 columns):
 #   Column           Dtype
---  ------           -----
 0   TRIPID           int32
 1   PROGRNUMBER      int8 
 2   STOPPOINTID      int16
 3   PLANNEDTIME_ARR  int32
 4   ACTUALTIME_ARR   int32
 5   DOY              int16
dtypes: int16(2), int32(3), int8(1)
memory usage: 1.9 GB


In [29]:
%%time
leaveTimes.duplicated().sum()

CPU times: user 1min 49s, sys: 36.7 s, total: 2min 26s
Wall time: 2min 26s


0

In [51]:
leaveTimes.nunique()

TRIPID             658961
PROGRNUMBER           109
STOPPOINTID          4774
PLANNEDTIME_ARR     72712
ACTUALTIME_ARR      74361
DOY                   360
dtype: int64

In [52]:
trips.nunique()

TRIPID             658964
LINEID                130
ROUTEID               588
DIRECTION               2
PLANNEDTIME_ARR     64461
PLANNEDTIME_DEP       791
ACTUALTIME_ARR      68122
ACTUALTIME_DEP      66771
DOY                   360
dtype: int64

## Merging

In [18]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182637 entries, 0 to 727498
Data columns (total 9 columns):
 #   Column           Dtype   
---  ------           -----   
 0   TRIPID           int32   
 1   LINEID           category
 2   ROUTEID          category
 3   DIRECTION        category
 4   PLANNEDTIME_ARR  float64 
 5   PLANNEDTIME_DEP  float64 
 6   ACTUALTIME_ARR   float64 
 7   ACTUALTIME_DEP   float64 
 8   DOY              int16   
dtypes: category(3), float64(4), int16(1), int32(1)
memory usage: 106.2 MB


In [19]:
trips.rename(columns = {'ACTUALTIME_ARR':'ACTUALTIME_ARR_T','PLANNEDTIME_ARR':'PLANNEDTIME_ARR_T', 'ACTUALTIME_DEP':'START_TIME', 'PLANNEDTIME_DEP':'PLANNED_START_TIME'}, inplace = True)


In [20]:
trips['LINEID'] = trips['LINEID'].astype(str)
trips['LINEID_DIRECTION'] = trips['LINEID'] + '_' + trips['DIRECTION'].astype(str)
trips = trips.drop(["DIRECTION", 'LINEID', "ROUTEID"], axis=1)
trips['LINEID_DIRECTION'] = trips['LINEID_DIRECTION'].astype(str)
trips.LINEID_DIRECTION = trips.LINEID_DIRECTION.str.replace(' ', '')

In [60]:
trips.LINEID_DIRECTION = trips.LINEID_DIRECTION.str.replace(' ', '')

In [21]:
trips['LINEID_DIRECTION'] = trips['LINEID_DIRECTION'].astype("category")


In [22]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182637 entries, 0 to 727498
Data columns (total 7 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PLANNEDTIME_ARR_T   float64 
 2   PLANNED_START_TIME  float64 
 3   ACTUALTIME_ARR_T    float64 
 4   START_TIME          float64 
 5   DOY                 int16   
 6   LINEID_DIRECTION    category
dtypes: category(1), float64(4), int16(1), int32(1)
memory usage: 99.9 MB


In [23]:
trips["PLANNED_START_TIME"] = trips["PLANNED_START_TIME"].astype("int32")

In [24]:
trips["START_TIME"] = trips["START_TIME"].fillna(-1)

In [25]:
trips["START_TIME"] = trips["START_TIME"].astype("int32")

In [26]:
trips = trips.drop(["PLANNEDTIME_ARR_T", "ACTUALTIME_ARR_T"], axis=1)

In [27]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2182637 entries, 0 to 727498
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PLANNED_START_TIME  int32   
 2   START_TIME          int32   
 3   DOY                 int16   
 4   LINEID_DIRECTION    category
dtypes: category(1), int16(1), int32(3)
memory usage: 50.0 MB


##### Saved ready trips file so there was no need to restart the entire process every time the kernel/server died.

In [28]:
# save to csv
trips.to_csv("./trips_lines_ready.csv", index=None)

In [30]:
#load csv
trips = pd.read_csv("trips_lines_ready.csv")

In [31]:
trips

,TRIPID,PLANNED_START_TIME,START_TIME,DOY,LINEID_DIRECTION
0,6253783,84600,84600,38,68_1
1,6262138,26460,-1,38,25B_2
2,6254942,32100,32082,38,45A_2
3,6259460,54420,54443,38,25A_1
4,6253175,81600,81608,38,14_1
...,...,...,...,...,...
2182632,6765849,57840,57859,134,123_2
2182633,6765469,48600,48823,134,75_1
2182634,6765486,26400,-1,134,33D_2
2182635,6764987,60600,-1,134,70_1


In [32]:
trips["TRIPID"] = trips["TRIPID"].astype("int32")
#trips["LINEID"] = trips["LINEID"].astype("category")
trips["DOY"] = trips["DOY"].astype("int16")
trips["LINEID_DIRECTION"] = trips["LINEID_DIRECTION"].astype("category")

In [33]:
trips["START_TIME"] = trips["START_TIME"].astype("int32")
trips["PLANNED_START_TIME"] = trips["PLANNED_START_TIME"].astype("int32")

In [34]:
trips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2182637 entries, 0 to 2182636
Data columns (total 5 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PLANNED_START_TIME  int32   
 2   START_TIME          int32   
 3   DOY                 int16   
 4   LINEID_DIRECTION    category
dtypes: category(1), int16(1), int32(3)
memory usage: 33.3 MB


- merge needed to add the direction, route id and line id

In [35]:
psutil.virtual_memory()

svmem(total=67481169920, available=17413992448, percent=74.2, used=41308098560, free=14256926720, active=36945543168, inactive=14771671040, buffers=748568576, cached=11167576064, shared=8072048640, slab=793923584)

In [36]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (6 entries)


In [37]:
psutil.virtual_memory()

svmem(total=67481169920, available=17419952128, percent=74.2, used=41302147072, free=14262812672, active=36937396224, inactive=14771695616, buffers=748576768, cached=11167633408, shared=8072048640, slab=793935872)

In [38]:
leaveTimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116949113 entries, 0 to 116949112
Data columns (total 6 columns):
 #   Column           Dtype
---  ------           -----
 0   TRIPID           int32
 1   PROGRNUMBER      int8 
 2   STOPPOINTID      int16
 3   PLANNEDTIME_ARR  int32
 4   ACTUALTIME_ARR   int32
 5   DOY              int16
dtypes: int16(2), int32(3), int8(1)
memory usage: 1.9 GB


In [39]:
%%time
# Upon merging it converts back to DASK sometimes(don't know why)
merged_dd = leaveTimes.merge(trips, how="left", on=["TRIPID", "DOY"])
#merged_dd.head(50)

CPU times: user 19.2 s, sys: 29.3 s, total: 48.5 s
Wall time: 48.6 s


In [42]:
merged_dd.shape

(116949113, 9)

In [40]:

psutil.virtual_memory()

svmem(total=67481169920, available=13338288128, percent=80.2, used=45383827456, free=10234019840, active=41019326464, inactive=14717861888, buffers=747622400, cached=11115700224, shared=8072048640, slab=793800704)

In [41]:
del leaveTimes
del trips

In [42]:
gc.collect()

0

In [43]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (3 entries)


In [44]:
psutil.virtual_memory()

svmem(total=67481169920, available=13258915840, percent=80.4, used=45463248896, free=10053029888, active=41100636160, inactive=14801383424, buffers=754524160, cached=11210366976, shared=8072056832, slab=799956992)

In [45]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,PLANNED_START_TIME,START_TIME,LINEID_DIRECTION
0,5972116,12,119,48030,48012,1,47400,47427,1_1
1,5966674,12,119,54001,54023,1,53400,53410,1_1
2,5959105,12,119,60001,59955,1,59400,59426,1_1
3,5966888,12,119,58801,58771,1,58200,58220,1_1
4,5965960,12,119,56401,56309,1,55800,55807,1_1
...,...,...,...,...,...,...,...,...,...
116949108,8588153,78,4383,28605,28998,365,25200,25257,27_2
116949109,8587459,78,4383,22695,23247,365,19800,19797,27_2
116949110,8586183,78,4383,51481,52237,365,46800,46810,27_2
116949111,8589374,23,7053,53659,53525,365,52200,-1,47_1


In [46]:
check =  merged_dd.loc[merged_dd.START_TIME == -1]

In [80]:
check.shape

(7869559, 10)

- no null values for start time

In [81]:
check.loc[check.PROGRNUMBER == 1]

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,PLANNED_START_TIME,START_TIME,ROUTEID_DIRECTION


In [47]:
del check
gc.collect()

0

In [48]:
merged_dd["start_error"] = merged_dd["START_TIME"] - merged_dd["PLANNED_START_TIME"]

In [49]:
merged_dd.loc[merged_dd.START_TIME != -1].start_error.mean()

35.993271067096586

In [84]:
merged_dd.loc[merged_dd.START_TIME != -1].start_error.min()

-11722

In [85]:
merged_dd.loc[merged_dd.START_TIME != -1].start_error.max()

17977

In [54]:
merged_dd = merged_dd.drop("start_error", axis=1)

In [50]:
merged_dd.shape

(116949113, 10)

In [50]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116949113 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PROGRNUMBER         int8    
 2   STOPPOINTID         int16   
 3   PLANNEDTIME_ARR     int32   
 4   ACTUALTIME_ARR      int32   
 5   DOY                 int16   
 6   PLANNED_START_TIME  int32   
 7   START_TIME          int32   
 8   LINEID_DIRECTION    category
dtypes: category(1), int16(2), int32(5), int8(1)
memory usage: 3.8 GB


In [51]:
%%time
merged_dd = merged_dd[merged_dd.START_TIME != -1]

In [52]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109079554 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PROGRNUMBER         int8    
 2   STOPPOINTID         int16   
 3   PLANNEDTIME_ARR     int32   
 4   ACTUALTIME_ARR      int32   
 5   DOY                 int16   
 6   PLANNED_START_TIME  int32   
 7   START_TIME          int32   
 8   LINEID_DIRECTION    category
dtypes: category(1), int16(2), int32(5), int8(1)
memory usage: 3.6 GB


In [67]:
merged_dd = merged_dd.drop("PLANNED_START_TIME", axis=1)

In [53]:
merged_dd["JOURNEY_TIME"] = 0

/tmp/ipykernel_8262/2160551051.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dd["JOURNEY_TIME"] = 0


In [54]:
merged_dd["JOURNEY_TIME"] = merged_dd["JOURNEY_TIME"].astype("int32")

/tmp/ipykernel_8262/863201917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dd["JOURNEY_TIME"] = merged_dd["JOURNEY_TIME"].astype("int32")


In [55]:
merged_dd["JOURNEY_TIME"] = merged_dd["ACTUALTIME_ARR"] - merged_dd["START_TIME"]

/tmp/ipykernel_8262/3531426695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dd["JOURNEY_TIME"] = merged_dd["ACTUALTIME_ARR"] - merged_dd["START_TIME"]


In [57]:
merged_dd["JOURNEY_TIME"].max()

23458

In [58]:
merged_dd.loc[merged_dd["JOURNEY_TIME"] > 20800]
# this is the day of pride parade

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,PLANNED_START_TIME,START_TIME,LINEID_DIRECTION,JOURNEY_TIME
63311828,7169852,21,278,55947,76884,196,54600,54751,1_2,22133
63312938,7169852,22,7615,56085,77118,196,54600,54751,1_2,22367
63312963,7169852,23,10,56103,77123,196,54600,54751,1_2,22372
63314072,7169852,24,12,56227,77195,196,54600,54751,1_2,22444
63314096,7169852,25,14,56282,77216,196,54600,54751,1_2,22465
63314118,7169852,26,15,56336,77294,196,54600,54751,1_2,22543
63315247,7169852,27,17,56415,77354,196,54600,54751,1_2,22603
63315272,7169852,28,18,56475,77421,196,54600,54751,1_2,22670
63316449,7169852,30,21,56558,77536,196,54600,54751,1_2,22785
63316499,7169852,32,85,56634,77651,196,54600,54751,1_2,22900


In [59]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,PLANNED_START_TIME,START_TIME,LINEID_DIRECTION,JOURNEY_TIME
0,5972116,12,119,48030,48012,1,47400,47427,1_1,585
1,5966674,12,119,54001,54023,1,53400,53410,1_1,613
2,5959105,12,119,60001,59955,1,59400,59426,1_1,529
3,5966888,12,119,58801,58771,1,58200,58220,1_1,551
4,5965960,12,119,56401,56309,1,55800,55807,1_1,502
...,...,...,...,...,...,...,...,...,...,...
116949107,8587465,78,4383,65040,65656,365,60600,60621,27_2,5035
116949108,8588153,78,4383,28605,28998,365,25200,25257,27_2,3741
116949109,8587459,78,4383,22695,23247,365,19800,19797,27_2,3450
116949110,8586183,78,4383,51481,52237,365,46800,46810,27_2,5427


In [61]:
merged_dd = merged_dd.drop(["START_TIME"], axis=1)

In [60]:
merged_dd["JOURNEY_TIME"] = merged_dd["JOURNEY_TIME"].astype("int16")

/tmp/ipykernel_8262/563391581.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dd["JOURNEY_TIME"] = merged_dd["JOURNEY_TIME"].astype("int16")


In [64]:
merged_dd["PLANNEDTIME_ARR"].max()

91680

In [68]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109079554 entries, 0 to 116949112
Data columns (total 8 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
dtypes: category(1), int16(3), int32(3), int8(1)
memory usage: 2.9 GB


In [171]:
# were all set to -1 at a later stage so this code won't run as it originally did
nulls_lt_a = merged_dd[merged_dd["ACTUALTIME_ARR"].isnull() == True]

In [172]:
nulls_lt_a

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH
3385959,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3385966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3385967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18651442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18651441,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81995537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58313705,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81995530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58313706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [173]:
nulls_lt_a.shape

(5891757, 14)

In [ ]:
#nulls_lt_a = nulls_lt_a[["ACTUALTIME_ARR_T", "PLANNEDTIME_ARR_T", "PROGRNUMBER", "DOY", "TRIPID", "ROUTEID_DIRECTION"]]
#nulls_lt_d = nulls_lt_d[["ACTUALTIME_DEP_T", "PLANNEDTIME_DEP_T", "PROGRNUMBER", "DOY", "TRIPID", "ROUTEID_DIRECTION"]]

In [ ]:
#nulls_lt_a

In [ ]:
#nulls_lt_a.info()

In [ ]:
#nulls_lt_d

In [ ]:
#null_routes = nulls_lt_a['ROUTEID_DIRECTION'].tolist()

In [ ]:
#null_routes[0:5]

In [ ]:
#nulls_lt_a.loc[nulls_lt_a["ROUTEID_DIRECTION"] == "1_37_1"]

In [ ]:
#nulls_lt_d.loc[nulls_lt_d["ROUTEID_DIRECTION"]== "33_71_2"]

In [ ]:
#route_check = nulls_lt_d.loc[nulls_lt_d["ROUTEID_DIRECTION"]== "33_71_2"]

In [ ]:
#del nulls_lt_a 
#del nulls_lt_d 

#gc.collect(nulls_lt_a)
#gc.collect(nulls_lt_d)

In [ ]:
#route_check.nunique()

- same 2 times but on 30 different days

In [ ]:
#route_33 = merged_dd.loc[merged_dd["ROUTEID_DIRECTION"]== "33_71_2"]
#route_33_1 = merged_dd.loc[merged_dd["ROUTEID_DIRECTION"]== "33_71_1"]

In [ ]:
#route_33

In [ ]:
#route_33_1

In [ ]:
#del route_33_1 
#del route_33 

#gc.collect(route_33)
#gc.collect(route_33_1)

- not every route goes both directions

In [ ]:
#check_1pn = merged_dd.loc[merged_dd["PROGRNUMBER"] == 1]

In [ ]:
#check_1pn

In [ ]:
#del check_1pn 


In [ ]:
#del null_routes

- trips data is same as progrnumber 1 data so columns can be dropped
- after considerations for the models we concluded there was no value in keeping the departure columns

In [65]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (8 entries)


In [55]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,PLANNED_START_TIME,START_TIME,LINEID_DIRECTION
0,5972116,12,119,48030,48012,1,47400,47427,1_1
1,5966674,12,119,54001,54023,1,53400,53410,1_1
2,5959105,12,119,60001,59955,1,59400,59426,1_1
3,5966888,12,119,58801,58771,1,58200,58220,1_1
4,5965960,12,119,56401,56309,1,55800,55807,1_1
...,...,...,...,...,...,...,...,...,...
116949108,8588153,78,4383,28605,28998,365,25200,25257,27_2
116949109,8587459,78,4383,22695,23247,365,19800,19797,27_2
116949110,8586183,78,4383,51481,52237,365,46800,46810,27_2
116949111,8589374,23,7053,53659,53525,365,52200,-1,47_1


In [58]:
type(merged_dd)

pandas.core.frame.DataFrame

In [60]:
merged_dd.shape

(109079554, 9)

In [56]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116949113 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column              Dtype   
---  ------              -----   
 0   TRIPID              int32   
 1   PROGRNUMBER         int8    
 2   STOPPOINTID         int16   
 3   PLANNEDTIME_ARR     int32   
 4   ACTUALTIME_ARR      int32   
 5   DOY                 int16   
 6   PLANNED_START_TIME  int32   
 7   START_TIME          int32   
 8   LINEID_DIRECTION    category
dtypes: category(1), int16(2), int32(5), int8(1)
memory usage: 3.8 GB


In [64]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dt            8759 non-null   int64         
 1   temp          8759 non-null   int64         
 2   wind_speed    8759 non-null   int64         
 3   weather_main  8759 non-null   int64         
 4   date          8759 non-null   datetime64[ns]
 5   DAYOFWEEK     8759 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 410.7 KB


In [65]:
weather.head(5)

,dt,temp,wind_speed,weather_main,date,DAYOFWEEK
0,1514768400,4,12,1,2018-01-01 01:00:00,0
1,1514772000,5,12,0,2018-01-01 02:00:00,0
2,1514775600,5,12,0,2018-01-01 03:00:00,0
3,1514779200,5,12,0,2018-01-01 04:00:00,0
4,1514782800,5,10,0,2018-01-01 05:00:00,0


In [66]:
weather['temp'] = weather['temp'].astype('int8')
weather['wind_speed'] = weather['wind_speed'].astype('int8')
weather['weather_main'] = weather['weather_main'].astype('int8')
weather['DAYOFWEEK'] = weather['DAYOFWEEK'].astype('int8')


In [67]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   dt            8759 non-null   int64         
 1   temp          8759 non-null   int8          
 2   wind_speed    8759 non-null   int8          
 3   weather_main  8759 non-null   int8          
 4   date          8759 non-null   datetime64[ns]
 5   DAYOFWEEK     8759 non-null   int8          
dtypes: datetime64[ns](1), int64(1), int8(4)
memory usage: 171.2 KB


###### remove nan values from buses that never left
- cant change the NaN values in dask so must do it in pandas when in individual files

In [68]:
def date_from_epoch(x):
    y = dt.datetime.fromtimestamp (x).strftime('%d-%m-%Y')
    return y   

weather['DAYOFSERVICE'] = weather['dt'].apply(lambda x: date_from_epoch(x))

def round_epoch_hour(x):
    day = x // 86400
    day = day * 86400
    time = x - day
    round_time = time // 3600
        
    return round_time

# weather['HOUR'] = weather['dt'].apply(lambda x: round_epoch_hour(x))

In [69]:
weather['dt'] = weather['dt'].astype("datetime64[s]")
weather['HOUR'] = weather['dt'].dt.hour


In [70]:
weather["HOUR"] = weather["HOUR"].astype("int8")

In [71]:
weather.head(5)

,dt,temp,wind_speed,weather_main,date,DAYOFWEEK,DAYOFSERVICE,HOUR
0,2018-01-01 01:00:00,4,12,1,2018-01-01 01:00:00,0,01-01-2018,1
1,2018-01-01 02:00:00,5,12,0,2018-01-01 02:00:00,0,01-01-2018,2
2,2018-01-01 03:00:00,5,12,0,2018-01-01 03:00:00,0,01-01-2018,3
3,2018-01-01 04:00:00,5,12,0,2018-01-01 04:00:00,0,01-01-2018,4
4,2018-01-01 05:00:00,5,10,0,2018-01-01 05:00:00,0,01-01-2018,5


In [72]:
weather["DAYOFSERVICE"] = weather["DAYOFSERVICE"].astype("datetime64")
weather["DOY"] = weather["DAYOFSERVICE"].dt.dayofyear

weather["DOY"] = weather["DOY"].astype("int16")


/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning: Parsing '13-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning: Parsing '14-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1063: UserWarning: Parsing '15-01-2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/pandas/co

In [73]:
weather = weather.drop({"dt", "date", "DAYOFSERVICE"}, axis=1)

In [6]:
psutil.virtual_memory()

svmem(total=67481169920, available=26719842304, percent=60.4, used=32007307264, free=18037485568, active=29638180864, inactive=17966735360, buffers=1880379392, cached=15555997696, shared=8072052736, slab=1141760000)

In [62]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (4 entries)


In [61]:
psutil.virtual_memory()

svmem(total=67481169920, available=22079606784, percent=67.3, used=44709974016, free=919740416, active=39221297152, inactive=25165090816, buffers=240136192, cached=21611319296, shared=7254016, slab=1501339648)

In [77]:
weather.dtypes

temp             int8
wind_speed       int8
weather_main     int8
DAYOFWEEK        int8
HOUR             int8
DOY             int16
dtype: object

In [ ]:
merged_dd.dtypes

In [10]:
#weather.to_csv(r'./weather_ready.csv', index=None)
weather = pd.read_csv(r'./weather_ready.csv')

In [11]:
weather["DOY"] = weather["DOY"].astype("int16")
weather['temp'] = weather['temp'].astype('int8')
weather['wind_speed'] = weather['wind_speed'].astype('int8')
#weather['weather_description'] = weather['weather_description'].astype('int8')
weather['weather_main'] = weather['weather_main'].astype('int8')
weather['DAYOFWEEK'] = weather['DAYOFWEEK'].astype('int8')
weather['HOUR'] = weather['HOUR'].astype('int8')

In [12]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8759 entries, 0 to 8758
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   temp          8759 non-null   int8 
 1   wind_speed    8759 non-null   int8 
 2   weather_main  8759 non-null   int8 
 3   DAYOFWEEK     8759 non-null   int8 
 4   HOUR          8759 non-null   int8 
 5   DOY           8759 non-null   int16
dtypes: int16(1), int8(5)
memory usage: 60.0 KB


In [17]:
for i in range(1,8):
    ttt = weather.loc[weather.DOY==i].head(1)
    print(ttt)

   temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
0     4          12             1          0     1    1
     temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
743     3           6             0          3     0    2
      temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
1415    -5          12             0          3     0    3
      temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
2158     2           2             0          5    23    4
      temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
2878     7           4             0          0    23    5
      temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
3622    13           4             0          3    23    6
      temp  wind_speed  weather_main  DAYOFWEEK  HOUR  DOY
4342    11           2             0          5    23    7


In [73]:
merged_dd.dtypes

TRIPID                 int32
PROGRNUMBER             int8
STOPPOINTID            int16
PLANNEDTIME_ARR        int32
ACTUALTIME_ARR         int32
DOY                    int16
LINEID_DIRECTION    category
JOURNEY_TIME           int16
dtype: object

In [74]:
psutil.virtual_memory()

svmem(total=67481169920, available=20515930112, percent=69.6, used=46273032192, free=5603491840, active=43969380352, inactive=15640268800, buffers=140562432, cached=15464083456, shared=7294976, slab=1588686848)

In [75]:
gc.collect()

0

In [77]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (6 entries)


In [ ]:
%%time

# used this way because it enabled noting when a time ran into the next calendar day
merged_dd['HOUR'] = merged_dd['ACTUALTIME_ARR'].apply(lambda row: round(row/3600))

#merged_dd['ACTUALTIME_ARR'] = merged_dd['ACTUALTIME_ARR'].astype("datetime64[s]")
#merged_dd['HOUR'] = merged_dd['ACTUALTIME_ARR'].dt.hour
#merged_dd['ACTUALTIME_ARR'] = merged_dd['ACTUALTIME_ARR'].values.astype("int32")

# standardised the method used to assign value for hour column and confirmed that there just happened to be no data at 3am
# this method suits best for making changes to dates 

In [79]:
merged_dd["HOUR"] = merged_dd["HOUR"].astype("int8")

In [ ]:
#W_HOUR = weather["HOUR"].unique()
#W_HOUR = W_HOUR.tolist()
#W_HOUR

In [ ]:
#m_hours = merged_dd["HOUR"].unique()
#m_hours = m_hours.tolist()
#m_hours

In [ ]:
#min_seconds = merged_dd.PLANNEDTIME_ARR.min()
#min_seconds // 3600

In [ ]:
#print(len(m_hours))
#print(len(W_HOUR))

# no 3am values 

In [80]:
merged_dd.loc[(merged_dd["HOUR"] == 24), "HOUR"] = 0
merged_dd.loc[(merged_dd["HOUR"] == 25), "HOUR"] = 1
merged_dd.loc[(merged_dd["HOUR"] == 26), "HOUR"] = 2

In [81]:
merged_dd.loc[(merged_dd["HOUR"] == 27), "HOUR"] = 3

In [82]:
# merged_dd.loc[(merged_dd["DAYOFWEEK"] == 7), "DAYOFWEEK"] = 0

In [83]:
merged_dd.loc[(merged_dd["HOUR"] == 0), "DOY"] += 1
merged_dd.loc[(merged_dd["HOUR"] == 1), "DOY"] += 1
merged_dd.loc[(merged_dd["HOUR"] == 2), "DOY"] += 1

In [84]:
merged_dd.loc[(merged_dd["HOUR"] == 3), "DOY"] += 1

In [85]:
merged_dd = merged_dd[merged_dd.DOY != 366]

In [86]:
m_hours = merged_dd["HOUR"].unique()
m_hours = m_hours.tolist()
m_hours

[13,
 15,
 17,
 16,
 20,
 21,
 11,
 12,
 10,
 18,
 19,
 23,
 22,
 14,
 9,
 0,
 8,
 1,
 7,
 6,
 5,
 2,
 4,
 3]

In [78]:
W_HOUR = weather["HOUR"].unique()
W_HOUR = W_HOUR.tolist()
W_HOUR

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 0]

In [87]:
merged_dd.HOUR.value_counts()

18    7571572
9     7121895
17    7094343
19    6976058
10    6926982
8     6769272
16    6651178
14    6643132
13    6514657
15    6501942
12    6324024
11    6263448
20    5612158
21    4870463
22    4661750
23    4612588
7     4556130
0     2405847
6      907410
1       52749
5       41894
2          51
4           8
3           2
Name: HOUR, dtype: int64

In [88]:
merged_dd.head(5)

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR
0,5972116,12,119,48030,48012,1,1_1,585,13
1,5966674,12,119,54001,54023,1,1_1,613,15
2,5959105,12,119,60001,59955,1,1_1,529,17
3,5966888,12,119,58801,58771,1,1_1,551,16
4,5965960,12,119,56401,56309,1,1_1,502,16


In [89]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109079553 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
dtypes: category(1), int16(3), int32(3), int8(2)
memory usage: 3.0 GB


In [81]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,JOURNEY_TIME,HOUR
0,5972116,12,119,48030,48012,1,1,1_37_1,585,13
1,5966674,12,119,54001,54023,1,1,1_37_1,613,15
2,5959105,12,119,60001,59955,1,1,1_37_1,529,17
3,5966888,12,119,58801,58771,1,1,1_37_1,551,16
4,5965960,12,119,56401,56309,1,1,1_37_1,502,16
...,...,...,...,...,...,...,...,...,...,...
116949107,8587465,78,4383,65040,65656,365,27,27_17_2,5035,18
116949108,8588153,78,4383,28605,28998,365,27,27_17_2,3741,8
116949109,8587459,78,4383,22695,23247,365,27,27_17_2,3450,6
116949110,8586183,78,4383,51481,52237,365,27,27_17_2,5427,15


In [90]:
merged_dd['HOUR'] = merged_dd['HOUR'].astype('int8')

In [91]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109079553 entries, 0 to 116949112
Data columns (total 9 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
dtypes: category(1), int16(3), int32(3), int8(2)
memory usage: 3.0 GB


In [ ]:
merged_dd.dtypes

In [ ]:
weather.dtypes

In [ ]:
merged_dd

In [ ]:
merged_dd.head(5)

In [ ]:
merged_dd.dtypes

In [ ]:
weather.dtypes

In [ ]:
merged_dd.head(3)

In [92]:
%%time
merged_dd = dd.merge(merged_dd, weather, on=['DOY', 'HOUR'], how='left')


CPU times: user 16.5 s, sys: 43.3 s, total: 59.8 s
Wall time: 59.9 s


In [93]:
psutil.virtual_memory()

svmem(total=67481169920, available=12641931264, percent=81.3, used=54143295488, free=2722881536, active=51746951168, inactive=10867847168, buffers=108089344, cached=10506903552, shared=7294976, slab=1456840704)

In [94]:
del weather

In [95]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (4 entries)


In [96]:
gc.collect()

0

In [97]:
psutil.virtual_memory()

svmem(total=67481169920, available=12727463936, percent=81.1, used=54057824256, free=2806325248, active=51660050432, inactive=10868441088, buffers=108113920, cached=10508906496, shared=7294976, slab=1456922624)

In [98]:
%%time
merged_dd = merged_dd.sort_values('LINEID_DIRECTION')

CPU times: user 23.3 s, sys: 40.7 s, total: 1min 3s
Wall time: 1min 4s


In [99]:
merged_dd.head(5)

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK
44885366,6755283,48,6015,36397,36143,138,102_1,2474,10,16.0,4.0,0.0,4.0
22744715,6393675,28,3636,32209,31724,78,102_1,971,9,2.0,8.0,0.0,0.0
22744714,6393675,27,3635,32193,31718,78,102_1,965,9,2.0,8.0,0.0,0.0
22744713,6393675,26,3634,32127,31674,78,102_1,921,9,2.0,8.0,0.0,0.0
22744712,6393675,25,4387,32096,31662,78,102_1,909,9,2.0,8.0,0.0,0.0


In [100]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109090240 entries, 44885366 to 47761184
Data columns (total 13 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
 9   temp              float64 
 10  wind_speed        float64 
 11  weather_main      float64 
 12  DAYOFWEEK         float64 
dtypes: category(1), float64(4), int16(3), int32(3), int8(2)
memory usage: 6.3 GB


In [101]:
merged_dd.isnull().sum(axis = 0)

TRIPID                  0
PROGRNUMBER             0
STOPPOINTID             0
PLANNEDTIME_ARR         0
ACTUALTIME_ARR          0
DOY                     0
LINEID_DIRECTION        0
JOURNEY_TIME            0
HOUR                    0
temp                11310
wind_speed          11310
weather_main        11310
DAYOFWEEK           11310
dtype: int64

In [97]:
merged_dd['temp'] = merged_dd['temp'].fillna(-1)
merged_dd['wind_speed'] = merged_dd['wind_speed'].fillna(-1)
merged_dd['weather_main'] = merged_dd['weather_main'].fillna(-1)
merged_dd['DAYOFWEEK'] = merged_dd['DAYOFWEEK'].fillna(-1)

In [ ]:
#nulls = merged_dd.loc[merged_dd['temp'].isnull()]

In [ ]:
#nulls.shape()

In [ ]:
#nulls

In [ ]:
weather.nunique()

In [ ]:
#nulls["DOY"].nunique()

In [ ]:
merged_dd["DOY"].nunique()

In [103]:
merged_dd.isnull().sum(axis = 0)

TRIPID              0
PROGRNUMBER         0
STOPPOINTID         0
PLANNEDTIME_ARR     0
ACTUALTIME_ARR      0
DOY                 0
LINEID_DIRECTION    0
JOURNEY_TIME        0
HOUR                0
temp                0
wind_speed          0
weather_main        0
DAYOFWEEK           0
dtype: int64

In [ ]:
#nulls["HOUR"].nunique()

In [94]:
merged_dd.shape

(109090241, 14)

In [105]:
merged_dd.DAYOFWEEK.value_counts()

0.0    17436133
3.0    16835539
2.0    16423176
1.0    16383440
4.0    16338889
5.0    13665015
6.0    11819393
Name: DAYOFWEEK, dtype: int64

#### all of the empty rows were on day 84 

In [106]:
merged_dd["DAYOFWEEK"] = merged_dd["DAYOFWEEK"].astype("int8")
merged_dd["temp"] = merged_dd["temp"].astype("int8")
merged_dd["wind_speed"] = merged_dd["wind_speed"].astype("int8")


In [107]:
merged_dd["weather_main"] = merged_dd["weather_main"].astype("int8")

In [102]:
merged_dd = merged_dd[merged_dd.DOY != 84]

In [104]:
merged_dd.loc[merged_dd.weather_main == -1]

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK


In [108]:
gc.collect()

0

In [109]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? Y
Flushing output cache (8 entries)


In [110]:
psutil.virtual_memory()

svmem(total=67481169920, available=8841568256, percent=86.9, used=57943003136, free=6241439744, active=54207856640, inactive=5505941504, buffers=89333760, cached=3207393280, shared=7294976, slab=830312448)

In [111]:
merged_dd["MONTH"] = 0

In [112]:
merged_dd["MONTH"] = merged_dd["MONTH"].astype("int8")

In [113]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108901585 entries, 44885366 to 47761184
Data columns (total 14 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
 9   temp              int8    
 10  wind_speed        int8    
 11  weather_main      int8    
 12  DAYOFWEEK         int8    
 13  MONTH             int8    
dtypes: category(1), int16(3), int32(3), int8(7)
memory usage: 3.5 GB


In [114]:
doys = merged_dd.DOY.unique()
doys

array([138,  78,  89,  88, 169, 170,  27,  28, 276, 275,  53, 243, 257,
       258, 117, 118,  59,  48, 256,  54, 136, 181, 323, 322,  49,  91,
       262, 248,  92,  10,  47,  11, 255, 182, 324, 128,  35,  36,  82,
       209,  81, 102, 221, 222, 210, 103, 112, 285, 113, 135, 176, 134,
       286, 274, 114, 184, 126, 277, 318, 180, 249, 110, 111,  73,  74,
       162, 282, 283, 175,  17,  44,  39, 150, 178, 179, 163,  26, 151,
        40, 316, 191,   2,  30,  41, 247, 250, 156, 314, 315, 312, 142,
        34, 143,  42, 101, 211, 158,  80, 232,  31,   3, 313, 100, 246,
       245, 196, 197, 130, 278, 185, 317,  29, 220, 132, 303, 251, 319,
       157, 125, 304,  93, 320,  38, 244, 119, 263, 148, 120,  56,  57,
       198,  45, 219, 152, 153, 107, 159,  97, 177, 195,  50, 131, 192,
       193,  85,  86, 212, 217,   9, 236, 237, 164, 165, 213, 330,  90,
       279, 329, 280, 265, 266, 291, 288, 287, 161,  16, 121, 331, 289,
       115, 133, 290, 116, 168,  43, 109, 144, 327, 328, 281,  1

In [115]:
%%time
for day in doys:
    #jan 31
    if day >=1 and day <=31:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 1
    #feb  28  
    if day > 31 and day <= 59:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 2
    #march  31  
    if day > 59 and day <= 90:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 3
    #april   30
    if day > 90 and day <= 120:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 4
    #may    31
    if day > 120 and day <= 151:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 5
    #june 30
    if day > 151 and day <= 181:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 6    
    #july    31
    if day >181 and day <= 212:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 7
    #august    31
    if day >212 and day <= 243:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 8
    #september  30 
    if day >243 and day <= 273:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 9
    #octo   31 
    if day > 273 and day <= 304:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 10
    #nov    30
    if day >304 and day <= 334:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 11
    #dec   31
    if day >334 and day <= 365:
        merged_dd["MONTH"].loc[merged_dd["DOY"] == day] = 12

    

<timed exec>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed ex

CPU times: user 36.6 s, sys: 1min 14s, total: 1min 50s
Wall time: 1min 51s


In [116]:
merged_dd.tail()

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH
64880218,7318116,22,1344,51550,51823,213,9_2,1367,14,5,6,0,0,8
64880219,7318120,44,201,76914,76867,213,9_2,2468,21,4,8,0,0,8
64880220,7316672,44,201,71514,72365,213,9_2,3365,20,4,8,0,0,8
64880203,7316045,22,1344,31848,32075,213,9_2,1436,9,-1,4,0,0,8
47761184,6856584,49,28,56578,56926,159,9_2,3055,16,22,4,0,0,6


In [117]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108901585 entries, 44885366 to 47761184
Data columns (total 14 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
 9   temp              int8    
 10  wind_speed        int8    
 11  weather_main      int8    
 12  DAYOFWEEK         int8    
 13  MONTH             int8    
dtypes: category(1), int16(3), int32(3), int8(7)
memory usage: 3.5 GB


In [111]:
fortySixA1 = merged_dd.loc[merged_dd['ROUTEID_DIRECTION']=="46A_68_1"]

In [112]:
fortySixA1

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH
44370956,6658312,20,2007,29222,29446,123,46A,46A_68_1,1765,8,4,2,0,0,5
77731480,7498790,22,2009,28006,27905,242,46A,46A_68_1,1561,8,12,2,0,3,8
16015899,6280265,22,2009,29278,29426,50,46A,46A_68_1,1801,8,9,4,0,0,2
16015900,6280265,27,2013,29679,29924,50,46A,46A_68_1,2299,8,9,4,0,0,2
50673628,6853274,44,2034,30806,30642,156,46A,46A_68_1,3061,9,15,4,0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7539811,6215245,43,2033,30520,30717,25,46A,46A_68_1,3081,9,5,6,1,3,1
44743805,6653011,1,264,26400,26455,124,46A,46A_68_1,0,7,0,2,0,3,5
44743806,6658312,2,6059,27718,27724,124,46A,46A_68_1,82,8,3,0,0,3,5
17779497,6286039,43,2033,30892,31253,58,46A,46A_68_1,3653,9,0,6,0,1,2


In [ ]:
#merged_dd

# make the task number smaller
- starting on 9506 for fortysixa1 and 8978 for merged_dd 

In [ ]:
print(type(fortySixA1))

- converts to pandas

In [113]:
fortySixA1.to_csv('./test_46a_dask_2.csv', index=False)  

In [114]:
del fortySixA1

In [ ]:
# merged_dd = pd.merge_asof(merged_dd, weather, on="col_name", direction='nearest')

In [ ]:
#merged_dd['epoch'] = merged_dd.DAYOFSERVICE.values.astype(np.int64).compute()

In [ ]:
#merged_dd['DAYOFSERVICE'] = merged_dd.DAYOFSERVICE.dt.day_name()
#merged_dd['DAYOFSERVICE'] = pd.factorize(merged_dd['DAYOFSERVICE'], sort=True)[0] # wasn't right


# idea is to create an extra column that rounds to the most recent hour in epoch form and join weather on that 

In [ ]:
# safety cell to make sure code below doens't run
printsdfgdsdg

### code for when ready to export file
- must sort the weather join first

In [118]:
routes = merged_dd.LINEID_DIRECTION.unique()
routes = sorted(routes)
print(len(routes))
print(routes)

252
['102_1', '102_2', '104_1', '104_2', '111_1', '111_2', '114_1', '114_2', '116_1', '116_2', '118_2', '11_1', '11_2', '120_1', '120_2', '122_1', '122_2', '123_1', '123_2', '130_1', '130_2', '13_1', '13_2', '140_1', '140_2', '142_1', '142_2', '145_1', '145_2', '14C_1', '14C_2', '14_1', '14_2', '150_1', '150_2', '151_1', '151_2', '15A_1', '15A_2', '15B_1', '15B_2', '15D_1', '15D_2', '15_1', '15_2', '161_1', '161_2', '16C_1', '16C_2', '16D_1', '16_1', '16_2', '17A_1', '17A_2', '17_1', '17_2', '184_1', '184_2', '185_1', '185_2', '18_1', '18_2', '1_1', '1_2', '220_1', '220_2', '236_1', '236_2', '238_1', '238_2', '239_1', '239_2', '25A_1', '25A_2', '25B_1', '25B_2', '25D_1', '25D_2', '25X_1', '25X_2', '25_1', '25_2', '26_1', '26_2', '270_1', '270_2', '27A_1', '27A_2', '27B_1', '27B_2', '27X_1', '27X_2', '27_1', '27_2', '29A_1', '29A_2', '31A_1', '31A_2', '31B_1', '31B_2', '31D_1', '31D_2', '31_1', '31_2', '32X_1', '32X_2', '32_1', '32_2', '33A_1', '33A_2', '33B_1', '33B_2', '33D_1', '33D_2

In [124]:
%%time
merged_dd["error"] = merged_dd["ACTUALTIME_ARR"] - merged_dd["PLANNEDTIME_ARR"]

CPU times: user 440 ms, sys: 1.55 s, total: 1.99 s
Wall time: 1.99 s


In [128]:
merged_dd["error"] = merged_dd["error"].astype("int16")

In [129]:
merged_dd["error"].min()

-11722

In [130]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108901585 entries, 44885366 to 47761184
Data columns (total 15 columns):
 #   Column            Dtype   
---  ------            -----   
 0   TRIPID            int32   
 1   PROGRNUMBER       int8    
 2   STOPPOINTID       int16   
 3   PLANNEDTIME_ARR   int32   
 4   ACTUALTIME_ARR    int32   
 5   DOY               int16   
 6   LINEID_DIRECTION  category
 7   JOURNEY_TIME      int16   
 8   HOUR              int8    
 9   temp              int8    
 10  wind_speed        int8    
 11  weather_main      int8    
 12  DAYOFWEEK         int8    
 13  MONTH             int8    
 14  error             int16   
dtypes: category(1), int16(4), int32(3), int8(7)
memory usage: 3.8 GB


In [92]:
merged_dd.head(100)

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH,error
18788532,6292706,23,945,26993,26768,57,102,102_10_2,7,1,4,0,0,2,-225
57284980,7100362,18,3608,27245,27131,179,102,102_10_2,8,21,4,0,3,6,-114
57284981,7102146,19,3609,25200,25133,179,102,102_10_2,7,19,4,0,3,6,-67
57284982,7101506,20,3598,26751,26826,179,102,102_10_2,7,19,4,0,3,6,75
57284983,7101506,21,4465,26778,26849,179,102,102_10_2,7,19,4,0,3,6,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25677854,6392242,3,3642,25962,25835,82,102,102_10_2,7,6,12,0,4,3,-127
25677852,6388408,2,3641,30059,29988,82,102,102_10_2,8,6,10,0,4,3,-71
25677848,6388565,1,1073,24300,24289,82,102,102_10_2,7,6,12,0,4,3,-11
38021114,6646200,24,947,31500,31482,116,102,102_10_2,9,9,10,0,3,4,-18


In [118]:
psutil.virtual_memory()

svmem(total=67481169920, available=14447616000, percent=78.6, used=52342525952, free=11008217088, active=49274720256, inactive=5814358016, buffers=23523328, cached=4106903552, shared=7303168, slab=681295872)

In [116]:
psutil.users()

[suser(name='team9', terminal='pts/0', host='185.134.146.112', started=1659502080.0, pid=23617),
 suser(name='team9', terminal='pts/19', host='185.134.146.112', started=1659502336.0, pid=24224),
 suser(name='team10', terminal='pts/31', host='tmux(22475).%0', started=1657274368.0, pid=22475)]

In [117]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (12 entries)


In [177]:
print(len(tops))
print(len(bottoms))

588
588


In [126]:
gc.collect()

898

In [150]:
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116759068 entries, 18788532 to 102464628
Data columns (total 16 columns):
 #   Column             Dtype   
---  ------             -----   
 0   TRIPID             int32   
 1   PROGRNUMBER        int8    
 2   STOPPOINTID        int16   
 3   PLANNEDTIME_ARR    int32   
 4   ACTUALTIME_ARR     int32   
 5   DOY                int16   
 6   LINEID             category
 7   ROUTEID_DIRECTION  category
 8   JOURNEY_TIME       int32   
 9   HOUR               int8    
 10  temp               int8    
 11  wind_speed         int8    
 12  weather_main       int8    
 13  DAYOFWEEK          int8    
 14  MONTH              int8    
 15  error              int32   
dtypes: category(2), int16(2), int32(5), int8(7)
memory usage: 4.7 GB


In [159]:
merged_dd.isna().sum()

TRIPID               0
PROGRNUMBER          0
STOPPOINTID          0
PLANNEDTIME_ARR      0
ACTUALTIME_ARR       0
DOY                  0
LINEID               0
ROUTEID_DIRECTION    0
HOUR                 0
temp                 0
wind_speed           0
weather_main         0
DAYOFWEEK            0
MONTH                0
error                0
dtype: int64

In [158]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH,error
18788532,6292706,23,945,26993,26768,57,102,102_10_2,7,1,4,0,0,2,-225
57284980,7100362,18,3608,27245,27131,179,102,102_10_2,8,21,4,0,3,6,-114
57284981,7102146,19,3609,25200,25133,179,102,102_10_2,7,19,4,0,3,6,-67
57284982,7101506,20,3598,26751,26826,179,102,102_10_2,7,19,4,0,3,6,75
57284983,7101506,21,4465,26778,26849,179,102,102_10_2,7,19,4,0,3,6,71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3654173,6099812,31,7589,84065,83931,12,9,9_8_2,23,9,0,0,5,1,-134
29651945,6596244,15,2437,84457,84473,93,9,9_8_2,23,4,2,0,6,4,16
3654172,6094861,31,7589,85265,85115,13,9,9_8_2,0,8,12,0,5,1,-150
3654147,6094861,17,2439,84547,84485,12,9,9_8_2,23,9,0,0,5,1,-62


In [127]:
%reset Out

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (12 entries)


In [108]:
print(min(tops))
print(max(tops))

-22.07353553164876
3154.1208962362184


In [109]:
print(min(bottoms))
print(max(bottoms))

-3498.383525496342
76.15273094053907


In [127]:
psutil.virtual_memory()

svmem(total=67481169920, available=12972294144, percent=80.8, used=53817020416, free=1413099520, active=50867847168, inactive=13583167488, buffers=86888448, cached=12164161536, shared=7331840, slab=895062016)

In [128]:
merged_dd = merged_dd.drop(["error"], axis=1)

In [129]:
# datatypes all reset for some reason
merged_dd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108901585 entries, 84054145 to 144058
Data columns (total 15 columns):
 #   Column             Dtype   
---  ------             -----   
 0   TRIPID             int32   
 1   PROGRNUMBER        int8    
 2   STOPPOINTID        int16   
 3   PLANNEDTIME_ARR    int32   
 4   ACTUALTIME_ARR     int32   
 5   DOY                int16   
 6   LINEID             category
 7   ROUTEID_DIRECTION  category
 8   JOURNEY_TIME       int32   
 9   HOUR               int8    
 10  temp               int8    
 11  wind_speed         int8    
 12  weather_main       int8    
 13  DAYOFWEEK          int8    
 14  MONTH              int8    
dtypes: category(2), int16(2), int32(4), int8(7)
memory usage: 4.0 GB


In [95]:
merged_dd.isnull().sum()
# i think it just ended uo making them nulls instead of dropping
# should check tomorrow

TRIPID               0
PROGRNUMBER          0
STOPPOINTID          0
PLANNEDTIME_ARR      0
ACTUALTIME_ARR       0
DOY                  0
LINEID               0
ROUTEID_DIRECTION    0
HOUR                 0
temp                 0
wind_speed           0
weather_main         0
DAYOFWEEK            0
MONTH                0
error                0
dtype: int64

In [176]:
merged_dd = merged_dd.dropna(how='any',axis=0) 

In [134]:
merged_dd

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH,error
44885366,6755283,48,6015,36397,36143,138,102_1,2474,10,16,4,0,4,5,-254
22744715,6393675,28,3636,32209,31724,78,102_1,971,9,2,8,0,0,3,-485
22744714,6393675,27,3635,32193,31718,78,102_1,965,9,2,8,0,0,3,-475
22744713,6393675,26,3634,32127,31674,78,102_1,921,9,2,8,0,0,3,-453
22744712,6393675,25,4387,32096,31662,78,102_1,909,9,2,8,0,0,3,-434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64880218,7318116,22,1344,51550,51823,213,9_2,1367,14,5,6,0,0,8,273
64880219,7318120,44,201,76914,76867,213,9_2,2468,21,4,8,0,0,8,-47
64880220,7316672,44,201,71514,72365,213,9_2,3365,20,4,8,0,0,8,851
64880203,7316045,22,1344,31848,32075,213,9_2,1436,9,-1,4,0,0,8,227


In [164]:
cd ..

/home/team9/tmp/data


In [119]:
ls

Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
leavetimes/
routes_no_outliers/
rt_leavetimes_DB_2018.txt
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
test_46a_dask_2.csv
test_46a_dask.csv
trips_lines_ready.csv
trips_modelling.csv
trips_ready.csv
weather_factorised.csv
weather_ready.csv
weather_trips.csv


In [120]:
mkdir lines_no_outliers

- pretty sure lines is the correct one to use but could look into it further

#### breaks up database into differenct files, divided by the lineid_direction column and cleans outlier rows by dropping all rows with double the value of the standard deviation

In [3]:
%%time

        
saved = []   
count = 0

for r in routes:
    count +=1
    temp_rows = merged_dd.loc[(merged_dd['LINEID_DIRECTION'] == r)]

    topTwoSDs = temp_rows.error.mean() + (2 * temp_rows.error.std())
    bottomTwoSDs = temp_rows.error.mean() - (2 * temp_rows.error.std())
    
    temp_rows = temp_rows.loc[(temp_rows.error < topTwoSDs)]
    temp_rows = temp_rows.loc[(temp_rows.error > bottomTwoSDs)]
    
    #temp_rows = temp_rows.drop("error", axis=1)

   #line_folder = temp_rows["LINEID"]

    name = f"line_{r}"

    temp_rows.to_csv(f'./lines_no_outliers/{name}.csv')
    saved.append(r)
    
    if count == 35:
        #del line_folder
        del temp_rows
        gc.collect()
        %reset Out
        count = 0
        print("save: ", saved)
        

        
    
    

NameError: name 'routes' is not defined

In [4]:
gc.collect()
%reset Out



Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Flushing output cache (1 entries)


In [5]:
psutil.virtual_memory()

svmem(total=67481169920, available=44201095168, percent=34.5, used=22587236352, free=34363006976, active=18952704000, inactive=12595400704, buffers=109391872, cached=10421534720, shared=7294976, slab=929488896)

In [135]:
temp_rows

,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,ACTUALTIME_ARR,DOY,LINEID,ROUTEID_DIRECTION,JOURNEY_TIME,HOUR,temp,wind_speed,weather_main,DAYOFWEEK,MONTH,error
13818033,6254321,26,1348,83685,83592,43,9,9_8_2,842,23,10,10,1,6,2,-93
36129535,6645077,10,2432,83092,83132,115,9,9_8_2,299,23,7,6,0,1,4,40
42769528,6744556,36,281,84475,84774,132,9,9_8_2,1953,0,6,6,1,2,5,299
56989546,7138311,23,1344,84442,84381,188,9,9_8_2,702,23,15,0,0,4,7,-61
66727254,7322643,19,2441,83343,83439,220,9,9_8_2,622,23,13,2,0,1,8,96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22881913,6391876,10,2432,83076,83079,79,9,9_8_2,288,23,3,4,0,1,3,3
22881914,6391876,11,2433,83111,83102,79,9,9_8_2,311,23,3,4,0,1,3,-9
58153439,7152067,20,2442,84637,84706,193,9,9_8_2,699,0,13,8,1,4,7,69
78491345,8084757,30,1353,85079,84784,263,9,9_8_2,788,0,11,4,0,3,9,-295
